In [15]:
# Input info
indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh/best_params/"
se_cells_meta_f = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_simpleUnion/knn/kparam_30/gff_A2_black/annotation_clones/se_cells_meta_labels.tsv"
#indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh/donor_0_thresh_results.tsv"

outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_simpleUnion/mt_clones_thresh/clonal_shifts/"


atac_col = "cluster_labels"
clone_col = "Variants"

# af_t = 0.1
# oth_af_t = 0.1
# cov_t = 10
# oth_cov_t = 10
# ncells = 10
# oth_ncells = 0.25 
# mean_pos_cov = 0

# config
N_DONORS = 2
input_cond = "Input"
condition = "inputOnly" # noInput

# params
min_clone_size = 10
p_thresh = 0.1 


#conds_sep = False
n_shuffle = 1000

n_cpus = 16

#se_cells_meta_f = /data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/gff_A2_black/annotation_clones/se_cells_meta_labels.tsv -p outdir /data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/variants_init/clonal_shifts/dendro_bc/results/inputOnly/knn/kparam_3/dendro_dt_0.6 -p barcodes_dir /data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/barcodes/btwnClones_dendro_dt_0.6
#N_DONORS = 2 
#condition = inputOnly


In [16]:
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from src import clonal_shifts as cs


In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
sep_dir = join(outdir,"sepDonors")
if not exists(sep_dir):
    os.mkdir(sep_dir)
donor_out = {}
for d in np.arange(N_DONORS):
    donor_out[d] = join(sep_dir, f"donor{d}")
    if not exists(donor_out[d]):
        os.mkdir(donor_out[d])
#     if not exists(join(donor_out[d], "shuffle")):
#         os.mkdir(join(donor_out[d], "shuffle"))

In [19]:
# cells_meta = cells_meta.loc[cells_meta["condition"]==input_cond]
# cells_meta.head()

In [20]:
bin_d = {}
best_params = pd.read_csv(join(indir,  "best_params.csv"))
for d in range(N_DONORS): 
    bin_d[d] = pd.read_csv(join(indir,  f"donor_{d}_binary.csv"), index_col=0)

labels_df = pd.read_csv(se_cells_meta_f,sep="\t").set_index("ID")
print(labels_df.shape)

labels_df = labels_df[~(labels_df["donor"]=='None')]
print(labels_df.shape)
labels_df.head()


# def fill_mt_bin(curr_pos, curr_cells):
#     print(curr_pos.name)
#     curr_pos.loc[curr_cells[curr_pos.name]["other_cells"]] = 0
#     curr_pos.loc[curr_cells[curr_pos.name]["clone_cells"]] = 1
#     return curr_pos

# cells_dir=join(indir, "cells")


# bin_d = {}
# for d in range(N_DONORS): 
#     curr_f = join(cells_dir, f"don.{d}_af.{af_t}_othaf.{oth_af_t}_cov.{cov_t}_othcov.{oth_cov_t}.p")
#     curr_cells = pickle.load(open(curr_f,'rb'))
#     params_results = pd.read_csv(join(indir, f"donor_{d}_thresh_results.tsv"), sep="\t")    


#     curr_p = params_results.loc[(params_results["af"]==af_t) &
#                        (params_results["oth_af"]==oth_af_t) &
#                        (params_results["cov"]==cov_t) &
#                        (params_results["oth_cov"]==oth_cov_t) &
#                        (params_results["ncells"]==ncells) &
#                        (params_results["oth_ncells"]==oth_ncells) &
#                        (params_results["mean_cov"]==mean_pos_cov)]
#     assert(len(curr_p)==1)
    
#     ## Construct a binary cell-by-variant matrix for the kept variants. 
#     ## Can add N/A if not in oth cells
#     curr_vars = curr_p.iloc[0]["Variants"].split(";")

#     all_cells = set()
#     for x in curr_cells:
#         all_cells = all_cells.union(set(curr_cells[x]["clone_cells"])) 
#         all_cells = all_cells.union(set(curr_cells[x]["other_cells"])) 

#     mt_bin = pd.DataFrame(index = all_cells, columns=curr_vars)
#     bin_d[d] = mt_bin.apply(fill_mt_bin, curr_cells=curr_cells,axis=0).fillna(0)



(18672, 39)
(16639, 39)


,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,...,ATAC_snn_res.0.8,seurat_clusters,nCount_RNA,nFeature_RNA,Row.names,lineage,donor,condition,name,cluster_labels
ID,,,,,,,,,,,,,,,,,,,,,
AAACGAAAGAATCAGT-1_Control,Control,1972,1972,6432,1259,31,19,266,1066,3791,...,6,6,2211,1942,Control_AAACGAAAGAATCAGT-1,4,1,Control,1_4,eosonophil
AAACGAAAGCATACCT-1_Control,Control,2674,2674,11858,1937,51,34,549,3102,6185,...,7,7,3311,2750,Control_AAACGAAAGCATACCT-1,30,1,Control,1_30,MPP/CMP/GMP C
AAACGAAAGCGAGAAA-1_Control,Control,1939,1939,7365,1097,25,27,275,2184,3757,...,7,7,2112,1838,Control_AAACGAAAGCGAGAAA-1,27,0,Control,0_27,MPP/CMP/GMP C
AAACGAAAGTACAGAT-1_Control,Control,862,862,4095,861,16,16,230,771,2201,...,4,4,1441,1317,Control_AAACGAAAGTACAGAT-1,4,0,Control,0_4,erythroid
AAACGAAAGTACAGTA-1_Control,Control,1775,1775,8088,1091,27,27,369,2157,4417,...,6,6,2385,2085,Control_AAACGAAAGTACAGTA-1,21,1,Control,1_21,eosonophil


# Filter for input

In [21]:
if condition == "inputOnly":
    labels_df = labels_df[labels_df["condition"]==input_cond]
elif condition == "noInput":
    labels_df = labels_df[labels_df["condition"]!=input_cond]
else:
    raise ValueError("condition not accurate")
for d in bin_d:
    bin_d[d] = bin_d[d].loc[bin_d[d].index.isin(labels_df.index)]
bin_d[d].head()

,10397G,10589A,11761T,13188T,146C,15297C,15497A,16129A,16355T,16362C,...,3847C,5442C,5581G,58C,64T,7598A,827G,8292A,8461T,9899C
AATGCCAGTCTGGGCT-1_Input,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
TAGCGGCGTCCTATTT-1_Input,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
TAAGTGCTCTATATCC-1_Input,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
TTGCGGGCAGTATCTG-1_Input,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
GCGATTAGTGGTCGAA-1_Input,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
def get_mt_ncells(curr_df, bin_d):
    lab, don = curr_df.name
    curr_bin = bin_d[int(don)]
    out = curr_bin.loc[curr_bin.index.isin(curr_df.index)].sum(axis=0).fillna(0)
    out.name = "count"
    return out #curr_bin.loc[curr_df.index].sum(axis=0)
    
lin_mt_ncells = labels_df.groupby(["cluster_labels", "donor"]).apply(get_mt_ncells, bin_d)
lin_mt_ncells = lin_mt_ncells.reset_index().rename({"level_2":"Variants"}, axis=1)

# convert back to raw cell count from log2
#lin_mt_ncells["count"] = (np.ceil((2**lin_mt_ncells["count"])-1)).astype(int)

In [14]:
pd.merge(labels_df[["donor", "cluster_labels", "condition"]], bin_d[d], 
         left_index=True, right_index=True, how="inner")


,donor,cluster_labels,condition,10397G,10589A,11761T,13188T,146C,15297C,15497A,...,3847C,5442C,5581G,58C,64T,7598A,827G,8292A,8461T,9899C
AAACGAAAGATATGAC-1_Input,1,MPP/CMP/GMP,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AAACGAAAGTAGTCGG-1_Input,1,MPP/CMP/GMP B,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AAACGAAGTGTTGTTG-1_Input,1,neutrophil,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AAACTCGCAAGTCTGT-1_Input,1,MPP/CMP/GMP,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AAACTCGCAGTGCTCG-1_Input,1,MEP,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTCAATGACTC-1_Input,1,MPP/CMP/GMP,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
TTTGTGTCAGAAAGAG-1_Input,1,MPP/CMP/GMP B,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
TTTGTGTCATAGCCAT-1_Input,1,MPP/CMP/GMP B,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
TTTGTGTGTCATAGAA-1_Input,1,MPP/CMP/GMP,Input,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
for d, val in lin_mt_ncells.groupby("donor"):
    #curr_groups = val.groupby([atac_col, clone_col]).size().reset_index().rename({0:"count"}, axis=1)
    curr_groups = val
    curr_sizes = val.groupby(clone_col).size().sort_values(ascending=False)
    curr_name_cond_size = val.groupby([clone_col]).size()
    curr_name_cond_size = curr_name_cond_size[curr_name_cond_size>min_clone_size]
    curr_clones_filt = curr_name_cond_size.index 
    curr_sizes = curr_sizes.loc[curr_clones_filt].sort_values(ascending=False)
    
    curr_clones = np.unique(curr_groups[clone_col])
    curr_clone_map = {x:ind for ind,x in enumerate(curr_clones)}
    print(curr_clones)
    curr_atac_cl = np.unique(curr_groups[atac_col])
    print(curr_atac_cl)

    curr_don_out = donor_out[int(d)]
    
    # Run init hypergeo and the counts data
    cs.hypergeo_plots(curr_groups, curr_clones, curr_atac_cl, curr_sizes, p_thresh, atac_col,
                   clone_col, curr_don_out)
    
    out_df, hyper_df, results_df, out_d = cs.run_data_and_shuffle(curr_groups, curr_don_out, atac_col, clone_col, p_thresh, curr_clones, 
                                                      curr_atac_cl, n_shuffle=n_shuffle, figs_close=False, to_p_correct=False,
                                                                  n_cpus=n_cpus)
    
#     output_df, bh_enrichment_df = cs.pipeline_groups_hypergeo(curr_groups, curr_clones, curr_atac_cl, curr_sizes,
#                                                              p_thresh=p_thresh, atac_col=atac_col, 
#                                                               clone_col=clone_col)
    
    #cs.pipeline_groups_hypergeo(curr_groups, curr_clones, curr_atac_cl, curr_sizes)
    
#     bh_enrichment_df.to_csv(join(curr_don_out, "hypergeo_input_padjusted.csv"))
#     output_df.to_csv(join(curr_don_out, "hypergeo_input_padjusted_sigOnly.csv"))
    
#     if output_df.shape[0] == 0:   
#         g = sns.clustermap(-np.log10(bh_enrichment_df.fillna(1)), 
#                        row_cluster=False)
#         g.fig.suptitle("No groups were significant")
#     else:
#         g = sns.clustermap(-np.log10(bh_enrichment_df.loc[output_df.index].fillna(1)), 
#                        row_cluster=False)
#     g.ax_heatmap.set(xlabel="Cluster ID")
#     g.ax_cbar.set(title="-log10 p-value")
#     plt.savefig(join(curr_don_out, "hypergeo_input_padjusted_sigOnly.png"))
    
    
#     init_bh_enrichment = cs.create_enrichment(curr_groups, atac_col, clone_col, p_thresh,
#                                               clones=curr_clones, atac_cl=curr_atac_cl)
    
#     shuffle = cs.shuffle_hypergeo(curr_groups, atac_col, clone_col, p_thresh, curr_clones, curr_atac_cl, 
#                                   n_shuffle=n_shuffle, to_parallel=True, n_cpus=24)

#     results_df, out_d = cs.get_out(shuffle, curr_clones, init_bh_enrichment, p_thresh, 
#                                                           curr_clone_map, atac_col, 
#                                                           outdir=join(curr_don_out, "shuffle"))
    
    
# #     # plot just the counts
# #     curr_groups["log2_count"] = np.log2(curr_groups["count"]+1)
# #     g = sns.clustermap(curr_groups.pivot(index=atac_col,columns=clone_col, values="log2_count").fillna(0))
# #     plt.gca().set_title("log2 ncells")
# #     plt.savefig(join(curr_don_out, "ncells.png"))
    

In [ ]:
out_df

In [ ]:
#cs.create_enrichment(curr_groups, atac_col, clone_col, p_thresh, clones=curr_clones, atac_cl=curr_atac_cl)
#     clones, atac_cl = get_groups(curr_groupsr, curr_clones, curr_atac_cl, clone_col,
#                                  atac_col)

In [ ]:
# for ind, val in results_df[results_df["value"]<p_thresh].groupby("method"):
#     sns.clustermap(val.astype(object).pivot(index="index", columns="variable",values="value").fillna(1),
#                   vmax=p_thresh+0.1)
#     plt.suptitle(ind)